In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo
import matplotlib.pyplot as plt
plt.style.use('seaborn-v0_8-muted')
from matplotlib import cm
import geopandas as gpd

from pyproj import Transformer
from concurrent.futures import ProcessPoolExecutor
import concurrent.futures
import multiprocessing
from multiprocessing import Pool
from functools import partial
import numpy.random as npr
import matplotlib.dates as mdates
from itertools import product
import copy
import pickle
from tqdm import tqdm

import nomad.io.base as loader
import nomad.city_gen as cg
from nomad.city_gen import City, Building
import nomad.traj_gen as tg
from nomad.traj_gen import Agent, Population
import nomad.stop_detection.ta_dbscan as DBSCAN
import nomad.stop_detection.lachesis as Lachesis

In [2]:
city_geojson = gpd.read_file('garden_city.geojson')
city = cg.load('garden-city.pkl')

### Generate N agents

The following code maps our Garden City coordinates to a location in the Atlantic Ocean (Atlantis?).

In [3]:
def garden_city_to_lat_long(agent, sparse_traj=True, full_traj=False, diaries=True):
    def project_city_blocks_to_web_mercator(df):
        """Convert (x, y) from 15m block units to Web Mercator meters via affine shift and projection."""
        transformer = Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)
        df['x'] = 15 * df['x'] + 4265699
        df['y'] = 15 * df['y'] - 4392976
        if 'ha' in df:
            df['ha'] = 15 * df['ha']
        df['longitude'], df['latitude'] = transformer.transform(df['x'].values, df['y'].values)
        df['date'] = df['local_timestamp'].dt.date
        return df

    def finalize(df):
        front = ['identifier', 'unix_timestamp', 'longitude', 'latitude', 'x', 'y', 'date']
        cols = [col for col in front if col in df] + [col for col in df.columns if col not in front]
        return df[cols].rename(columns={'identifier': 'uid', 'unix_timestamp': 'timestamp'}).reset_index(drop=True)

    if sparse_traj:
        agent.sparse_traj = finalize(project_city_blocks_to_web_mercator(agent.sparse_traj))
    if full_traj:
        agent.trajectory = finalize(project_city_blocks_to_web_mercator(agent.trajectory))
        
    if diaries:
        diary = agent.diary.copy()
        xs = []
        ys = []
        for loc in diary["location"]:
            if loc is None:
                xs.append(None)
                ys.append(None)
            else:
                pt = agent.city.buildings[loc].geometry.centroid
                xs.append(pt.x)
                ys.append(pt.y)
        diary["x"] = xs
        diary["y"] = ys
        agent.diary = finalize(project_city_blocks_to_web_mercator(diary))

Initiate $N$ empty agents. 

In [5]:
N = 150  # can modify

population = Population(city)
population.generate_agents(N=N, seed=16, name_count=2)

## Simple trajectory generation

For simple trajectory generation tasks that don't require too much computation power and can be done on a personal laptop, the following code generates a trajectory for each agent and saves it to a csv.

In [6]:
def generate_agent_trajectory(agent_id, agent, seed):
    
    beta_duration = npr.uniform(12, 150)
    beta_start = max(npr.uniform(30, 600), beta_duration)
    beta_ping = min(npr.uniform(3, 20), beta_duration//2)
    print(beta_ping, beta_start, beta_duration)
    agent.generate_trajectory(
        local_timestamp = "2024-01-01T06:00 +02:00",
        end_time = pd.Timestamp('2024-01-08T13:30:00 +02:00'),
        seed=100,
        dt=0.5)
    print('finished generating trajectory')
    agent.sample_trajectory(
        beta_start=beta_start,
        beta_durations=beta_duration,
        beta_ping = beta_ping,
        seed=seed,
        replace_sparse_traj=True)
    
    garden_city_to_lat_long(agent,
                            sparse_traj=True,
                            full_traj=False)
    
    return agent_id, copy.deepcopy(agent)

In [7]:
agent_id, agent = [(agent_id, agent) for agent_id, agent in population.roster.items()][0]
agent_id

'adoring_keldysh'

In [8]:
generate_agent_trajectory(agent_id, agent, 150)

4.165468916947845 160.8500755093817 145.3669557058947
finished generating trajectory


('adoring_keldysh', <nomad.traj_gen.Agent at 0x2a92ab37790>)

In [9]:
def generate_trajectory_data(agent_id, agent, seed):
    agent.reset_trajectory()
    
    agent.generate_trajectory(
        local_timestamp="2024-01-01T06:00:00 +02:00",
        end_time=pd.Timestamp("2024-01-20T12:00:00 +02:00"),
        seed=105,
        dt=1)

    beta_duration = npr.uniform(25, 170)
    beta_start = max(npr.uniform(25, 520), beta_duration)
    beta_ping = min(npr.uniform(3, 15), beta_duration//2)

    agent.sample_trajectory(
        beta_start=beta_start,
        beta_durations=beta_duration,
        beta_ping=beta_ping,
        seed=seed,
        replace_sparse_traj=True)

    garden_city_to_lat_long(agent, sparse_traj=True, full_traj=False)
    return None

# Generate trajectories with progress bar
for agent_id, agent in tqdm(population.roster.items(), desc="Generating trajectories"):
    generate_trajectory_data(agent_id, agent, seed=150)

Generating trajectories: 100%|███████████████████████████████████████████████████████| 150/150 [07:20<00:00,  2.94s/it]


In [10]:
traj_cols = {
    "user_id": "uid",
    "timestamp": "timestamp",
    "latitude": "latitude",
    "longitude": "longitude",
    "x": "x",
    "y": "y",
    "duration": "duration",
    "datetime": "local_timestamp"}
# Save only sparse trajectories and diaries
population.save_pop(
    sparse_path="output/sparse_traj/",
    diaries_path="output/diaries/",
    partition_cols={
        "sparse_traj": ["date"],
        "diaries": ["uid"]
    },
    traj_cols=traj_cols
)

C:\Users\franc\Documents\Nomad\nomad-repo\nomad\io\base.py:55: UserWarning: Trajectory column 'duration' specified for 'duration' not found in DataFrame.
  warnings.warn(f"Trajectory column '{value}' specified for '{key}' not found in DataFrame.")


In [11]:
sparse_df = loader.from_file("output/sparse_traj/", format="parquet", traj_cols=traj_cols,
                      parse_dates=True)
diaries_df = loader.from_file("output/diaries/", format="parquet", traj_cols=traj_cols,
                       parse_dates=True)

In [12]:
diaries_df.head()

,timestamp,longitude,latitude,x,y,date,local_timestamp,duration,location,uid
0,1704081660,38.321380,-36.667407,4265916.5,-4392848.5,2024-01-01,2024-01-01 06:01:00+02:00,120,h-x13-y8,admiring_curie
1,1704088860,NaN,NaN,NaN,NaN,2024-01-01,2024-01-01 08:01:00+02:00,5,None,admiring_curie
2,1704089160,38.319696,-36.666921,4265729.0,-4392781.0,2024-01-01,2024-01-01 08:06:00+02:00,10,r-x3-y12,admiring_curie
3,1704089760,NaN,NaN,NaN,NaN,2024-01-01,2024-01-01 08:16:00+02:00,2,None,admiring_curie
4,1704089880,38.319696,-36.667083,4265729.0,-4392803.5,2024-01-01,2024-01-01 08:18:00+02:00,28,r-x3-y11,admiring_curie


In [13]:
sparse_df.head()

,uid,timestamp,longitude,latitude,x,y,local_timestamp,ha,date
0,inspiring_beaver,1704093480,38.320071,-36.666358,4.265771e+06,-4.392703e+06,2024-01-01 09:18:00+02:00,11.25,2024-01-01
1,inspiring_beaver,1704093720,38.319979,-36.666396,4.265761e+06,-4.392708e+06,2024-01-01 09:22:00+02:00,11.25,2024-01-01
2,inspiring_beaver,1704095220,38.320034,-36.666405,4.265767e+06,-4.392709e+06,2024-01-01 09:47:00+02:00,11.25,2024-01-01
3,inspiring_beaver,1704097140,38.320061,-36.666418,4.265770e+06,-4.392711e+06,2024-01-01 10:19:00+02:00,11.25,2024-01-01
4,inspiring_beaver,1704097440,38.320072,-36.666500,4.265771e+06,-4.392723e+06,2024-01-01 10:24:00+02:00,11.25,2024-01-01


## Parallelized Trajectory Generation

For larger trajectory generation tasks that require a lot of compute power, we can parallelize the trajectory generation using the following code. We generate ground-truth trajectories in agent-month "chunks", sparsify each chunk, then reset the ground-truth trajectory field to lessen the memory usage. 

In [ ]:
# Using parallel processing (e.g., using a cluster)
%%time

def generate_trajectory_data(agent_id, agent, seed):
    
    beta_duration = npr.uniform(15, 180)
    beta_start = max(npr.uniform(60, 1200), beta_duration*3)
    beta_ping = npr.uniform(1.5, 30)
    
    param = (beta_start, beta_duration, beta_ping)
    
    for month in range(1,13):
        days = calendar.monthrange(2024, month)[1]
        population_n.generate_trajectory(agent, 
                                         T=datetime(2024, month, days, hour=23, minute=59), 
                                         seed=seed)
    
        agent.sample_traj_hier_nhpp(*param, 
                                    seed=seed,
                                    reset_traj=True)
    
    garden_city_to_lat_long(agent,
                            sparse_traj=True,
                            full_traj=False)
    
    return agent_id, copy.deepcopy(agent)

manager = multiprocessing.Manager()
shared_roster = manager.dict(population_n.roster)

start = 6001  # 12001  # can modify
end = 12001   # 18001  # can modify
roster = dict(population_n.roster)
batch = islice(roster.items(), start, end)

with ProcessPoolExecutor() as executor:
    with tqdm(total=(end-start), desc="Processing agents") as pbar:
        futures = [
            executor.submit(generate_trajectory_data, agent_id, agent, i+15000)
            for i, (agent_id, agent) in enumerate(batch, start=start)
        ]
        results = []
        for future in futures:
            results.append(future.result())
            pbar.update(1)

for agent_id, agent in results:
    population_n.roster[agent_id] = agent

This code saves the generated trajectories in a parquet file, using the date as the partition column.

In [ ]:
partition_cols = {
    'sparse_traj': ['date'],
    'diaries': ['identifier']
}

roster = dict(islice(population_n.roster.items(), start, end))

population_n.save_pop(bucket="synthetic-raw-data",
                      prefix=f"agents-{start+15000}-{end+15000-1}/",
                      save_full_traj=False,
                      save_sparse_traj=True,
                      save_homes=True,
                      save_diaries=True,
                      partition_cols=partition_cols,
                      roster=roster)

In [ ]:
# Load the Parquet files

s3_path = "s3://synthetic-raw-data/agents-1-1001/sparse_trajectories.parquet/"
df1 = pd.read_parquet(s3_path)
s3_path = "s3://synthetic-raw-data/agents-1001-2000/sparse_trajectories.parquet/"
df2 = pd.read_parquet(s3_path)